In [2]:
import pandas as pd
import gensim
import multiprocessing
import numpy as np
from utils import pickle_obj, semantic_search_author, semantic_search_word, get_related_authors, get_related_words, translate_dict
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import HoverTool, ColumnDataSource, value
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds


In [3]:
df = pd.read_csv("assets/dataframes/all_authors_df_2004")
df_auth = pd.read_csv("assets/dataframes/KT_auth_2004")

In [4]:
df

,Unnamed: 0,Abstracts,Doc_id,KTH_name,KTH_id,KTH_name_id,department,school
0,0,A model for the enhanced water dissociation th...,65923903,"Dahlkild, Anders A.:Behm, Marten",u16fm297:u13dx9f6,NaN,Mekanik,Skolan för teknikvetenskap (SCI)
1,1,Wood from white spruce Picea glauca that had ...,40520967,"Zhang, Liming:Henriksson, Gunnar",u16k1pmb:u1lkf75c,NaN,Fiber- och polymerteknologi,"Skolan för kemi, bioteknologi och hälsa (CBH)"
2,2,"We consider the moduli space H-g,H-n of n-poin...",41290309,"Bergstrom, Jonas",u1mv0zlg,NaN,Matematik (Inst.),Skolan för teknikvetenskap (SCI)
3,3,In this paper a day-ahead planning algorithm f...,40129874,"Matevosyan, Julija:Olsson, Magnus:Soder, Lennart",u16u3erw:u1naf2f4:u1fjok0u,NaN,Elkraftteknik,Skolan för elektroteknik och datavetenskap (EECS)
4,4,"In this letter, we develop a fixed-point arith...",61385618,"Johansson, Christopher:Lansner, Anders B.",u1s42xk3:u12s8cr8,NaN,Beräkningsvetenskap och beräkningsteknik (CST),Skolan för elektroteknik och datavetenskap (EECS)
...,...,...,...,...,...,...,...,...
23806,23806,Purpose: This paper seeks ways to address posi...,80855494,"Ekener-Petersen, Elisabeth",u14iag3o,NaN,"Hållbar utveckling, miljövetenskap och teknik",Skolan för arkitektur och samhällsbyggnad (ABE)
23807,23807,"Many physical systems, such as water/electrici...",86902074,"Du, Rong:Gkatzikis, Lazaros:Flischione, Carlo:...",u1w7ca8o:u156rgrs:u18ax6vz:u1iq6n9a,NaN,Nätverk och systemteknik,Skolan för elektroteknik och datavetenskap (EECS)
23808,23808,"Wear processes from mechanical braking, rail/w...",80640634,"Cha, Yingying:Abbasi, Saeed:Olofsson, Ulf L-O",u1tg7qkt:u1l8376n:u10qpddv,NaN,Maskinkonstruktion (Inst.),Skolan för industriell teknik och management (...
23809,23809,Storage stability and phase separation behavio...,85192803,"Zhu, Jiqing:Kringos, Nicole",u17jpr0l:u1vwo6eu,NaN,Byggvetenskap,Skolan för arkitektur och samhällsbyggnad (ABE)


In [3]:
text_doc, doc_id, dep, auth, name = df.Abstracts.values, df.Doc_id, df.department, df.KTH_id, df.KTH_name

In [4]:
def read_corpus(abstracts, doc, auth):
    for d, w, a in zip(abstracts, doc, auth):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(d), [str(w)] + a.split(":"))

In [5]:
train_corpus = list(read_corpus(text_doc, doc_id, auth))

In [6]:
cores = multiprocessing.cpu_count()

model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=1, dm=0,
                                      sample=1e-3, negative=15,hs=0,dbow_words=1,
                                      max_vocab_size=None,workers=cores,window=10)

model.build_vocab(train_corpus)


I have pre-trained a model with vectordim=500, and with 10_000 iterations. The rest of the hyper-parameters are the same as in the implementation above.

In [7]:
#model.train(train_corpus, total_examples=model.corpus_count, epochs=1,report_delay=1)
model = gensim.models.Word2Vec.load("assets/doc2vecModels/KTH2004_i10000_w10_d500_plainTrain/KTH2004_i10000_w10_d500_plainTrain.model")

Import dictonaries

In [8]:
pickle_o = picbkle_obj()
id_to_auth = pickle_o.load("./assets/dictionaries/id_to_all_auths_2004")
auth_to_id = {k:v for v, k in id_to_auth.items()}

# Check the success of the training

## Semantic serach after authors.

In [24]:
sentence = ["dogs","forensic"]

In [25]:
semantic_search_author(sentence, model, df, topn=30)

44443867 || Klutsch, Cornelya F. C. and Savolainen, Peter  ||  0.37
78452380 || Savolainen, Peter  ||  0.35
48388983 || Green, Henrik  ||  0.33
1950 || Savolainen, Peter  ||  0.32
70652614 || Ardalan, Arman and Savolainen, Peter  ||  0.31
48173366 || Savolainen, Peter  ||  0.31
49468617 || Song, Yajing , Araujo, Ana Catarina , Lundberg, Joakim and Brumer, Harry, III  ||  0.31
48172766 || Vickovic, Sanja  ||  0.31
67681093 || Savolainen, Peter  ||  0.3
71432556 || Savolainen, Peter  ||  0.3
49049282 || Kahles, Andre , Savolainen, Peter and Arvestad, Lars  ||  0.3
67960998 || Lu, X.X.  ||  0.3
48249079 || Savolainen, Peter  ||  0.29
48241169 || Zhang, Ai-bing , Klutsch, Cornelya F. C. and Savolainen, Peter  ||  0.29
44153880 || Engkvist, Inga-Lill  ||  0.29
65740864 || Andrade, Jorge  ||  0.29
63752126 || Oskarsson, Mattias and Savolainen, Peter  ||  0.28
72170043 || Darmani, Anna  ||  0.28


## Semantic serach after authors.

In [26]:
semantic_search_word(sentence, model, df, topn=10)

('dog', 0.45107579231262207)
('mtdna', 0.44751986861228943)
('breeds', 0.40408754348754883)
('domestication', 0.34156543016433716)
('wolves', 0.340658962726593)
('autopsies', 0.3394320607185364)
('geneticists', 0.3360940217971802)
('autopsy', 0.33227604627609253)


## Search for related authors

In [15]:
get_related_authors("Kall, Lukas(u1gqsept)",model, auth_to_id, id_to_auth, topn=20)

The, Matthew 0.5885154008865356
Tasnim, Ayesha 0.3801935911178589
Jeuken, Gustavo S. 0.3241502046585083
Edfors, Fredrik 0.3126299977302551
Danielsson, Frida 0.2991322875022888
Lundberg, Emma K. 0.289335161447525


In [170]:
get_related_authors("Savolainen, Peter",model, auth_to_id, id_to_auth, topn=20)

Klutsch, Cornelya F. C. 0.763911783695221
Oskarsson, Mattias 0.7525292038917542
Ardalan, Arman 0.6701416373252869
Zhang, Ai-bing 0.6636019349098206
Angleby, Helen 0.6237658858299255
Ekstrom, Camilla 0.4878050982952118
Dahlgren, Lars-Goran 0.4533730447292328
Natanaelsson, Christian 0.3749474287033081
Skollermo, Anna 0.3562085032463074
Andersson, Anders  F. 0.33745628595352173
Arvestad, Lars 0.3230893611907959


## Search for related words

In [16]:
get_related_words("Kall, Lukas(u1gqsept)", model, auth_to_id, id_to_auth, topn=20)

('percolator', 0.5342718958854675)
('peptides', 0.5266672372817993)
('identifications', 0.47334036231040955)
('psms', 0.4226508140563965)
('shotgun', 0.41497480869293213)
('fdr', 0.3953171968460083)
('peptide', 0.3790532648563385)
('proteomics', 0.3485048711299896)
('decoy', 0.3277590274810791)
('diffacto', 0.3210739493370056)
('mascot', 0.3203628361225128)
('elude', 0.29521048069000244)
('proteome', 0.2931971549987793)
('fdrs', 0.29087352752685547)
('etd', 0.28902000188827515)
('spectrometry', 0.28638899326324463)
('psm', 0.28273484110832214)
('thermolysin', 0.28136274218559265)
('rna', 0.27758726477622986)
('clusterings', 0.2771516442298889)


In [19]:
get_related_words("Savolainen, Peter", model, auth_to_id, id_to_auth, topn=20)

('dogs', 0.8062396049499512)
('dog', 0.6066737174987793)
('mtdna', 0.544166624546051)
('breeds', 0.5020272731781006)
('domestication', 0.43833571672439575)
('wolves', 0.41998612880706787)
('asia', 0.4150415360927582)
('wolf', 0.41222769021987915)
('haplotypes', 0.37536320090293884)
('haplotype', 0.3406689167022705)
('southeast', 0.33729326725006104)
('crossbreeding', 0.33585119247436523)
('genetic', 0.3333330750465393)
('usingthe', 0.33036643266677856)
('eskimo', 0.328866183757782)
('alaskan', 0.32853883504867554)
('breed', 0.327038049697876)
('domesticates', 0.3199591636657715)
('kangals', 0.317442387342453)
('forensic', 0.31461000442504883)


In [144]:
get_related_words("Hudson, Paul", model, auth_to_id, id_to_auth, topn=20)

('synechocystis', 0.518214762210846)
('cyanobacteria', 0.4277106523513794)
('crispri', 0.39211180806159973)
('butanol', 0.37669068574905396)
('starvation', 0.3419566750526428)
('mep', 0.3409584164619446)
('polyclonal', 0.33996710181236267)
('metabolic', 0.33374160528182983)
('epitopes', 0.330426961183548)
('ribosome', 0.32393917441368103)
('overexpression', 0.3234694302082062)
('titers', 0.3067764639854431)
('repression', 0.30260127782821655)
('crispr', 0.30258870124816895)
('ribosomes', 0.2986229956150055)
('proteome', 0.2968001365661621)
('biosynthesis', 0.2954593598842621)
('optgene', 0.2952108383178711)
('cyanobacterium', 0.2941749691963196)
('monoclonal', 0.29319706559181213)


# Make a plot of the tSNE manifold to verify a underlying structure of the authorvectors

Get all authors-ids and its vectors

In [46]:
cat = list()
docVec = list()
for k, v in zip(model.docvecs.doctags.keys(), model.docvecs.doctag_syn0): 
    if k[0] == "u":
        cat.append(id_to_auth[k])
        docVec.append(v)
docVec = np.asarray(docVec)

/home/ekvall/anaconda3/envs/kth-cluster/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


Find what department author is a member of.

In [32]:
dep_names = list()
for n in cat: 
    if 0 < df_auth[df_auth.Name.str.contains(str(n))].dep_name.describe().unique()[0]:
        dep_names.append(df_auth[df_auth.Name.str.contains(str(n))].dep_name.describe().top)
    else:
        dep_names.append("nan")

/home/ekvall/anaconda3/envs/kth-cluster/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.


Make sure that the deprtment names have is trandlated into ascii

In [35]:
dep_names = np.asarray(dep_names)
name_dict = dict()
for u_id in np.unique(dep_names):
    n = str()
    if u_id is np.nan:
        u_id = str(u_id)
    for l in u_id:
        if ord(l) < 128:
            n +=l
        else:
            n += translate_dict[l]
    name_dict[str(u_id)] = str(n)
    
asci_cat = list()
for c in dep_names:asci_cat.append(name_dict[str(c)])

Make tSNE over author vectors

In [63]:
tsne_input = docVec
tsne_input = pd.DataFrame(tsne_input, index=asci_cat)
tsne_input = tsne_input
tsne = TSNE(perplexity=5)
%time tsne_vectors = tsne.fit_transform(tsne_input)
tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsne_input.index),
                            columns=[u'x_coord', u'y_coord'])
tsne_vectors[u'word'] = tsne_vectors.index

CPU times: user 1min 21s, sys: 59.9 ms, total: 1min 21s
Wall time: 1min 19s


Loading BokehJS ...

Make the plot

## The plot seems to have an underlying structure, check it out!

In [57]:
output_notebook()
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title=u't-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= (u'pan, wheel_zoom, box_zoom,'
                           u'box_select, reset'),
                   active_scroll=u'wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = u'@word') )

# draw the words as circles on the plot
tsne_plot.circle(u'x_coord', u'y_coord', source=plot_data,
                 color=u'blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color=u'black')


tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# engage!
show(tsne_plot);

In [62]:
output_file("./assets/figures/tSNEauthors.html")
save(tsne_plot)

'/home/ekvall/kthProject/getLifeScience/assets/figures/tSNEauthors.html'

# Testing some SVD

In [207]:
def get_U(V, comp=90):
    U, _, _ = svds(V, k=comp)
    norms = np.sqrt(np.sum(np.square(U), axis=1, keepdims=True))
    U /= np.maximum(norms, 1e-7)
    return U

In [208]:
U = get_U(model.wv.vectors)

In [209]:
words = model.wv.index2word
x2i = {w:i for i, w in enumerate(words)}
i2x = {i:w for i, w in enumerate(words)}

In [196]:
k = 5
for x in ['dogs', 'biofuels', 'peptide',"percolator","forensic","economic",
          "finance","algebra","convergence","bacteria"]:

    dd = cosine_similarity(U,U[x2i[x]].reshape(1,-1)).flatten()
    s = ''
    for i in np.argsort(dd)[::-1][:k + 1]:
        if i2x[i] == x: continue
        xy = tuple(sorted((x, i2x[i])))
        s += '(%s, %.3lf) ' % (i2x[i], dd[i])
    print('%s, %s' % (x, s))


dogs, (dog, 0.690) (breeds, 0.654) (mtdna, 0.633) (wolves, 0.595) (haplogroup, 0.534) 
biofuels, (arepresented, 0.485) (fqd, 0.483) (anp, 0.478) (lowcarbonenergy, 0.463) (prices, 0.458) 
peptide, (peptides, 0.669) (antibodies, 0.534) (protein, 0.512) (amino, 0.489) (antibody, 0.476) 
percolator, (shotgun, 0.544) (maxquant, 0.485) (uttermost, 0.462) (mascot, 0.440) (padlock, 0.439) 
forensic, (geneticists, 0.480) (fourthly, 0.434) (autopsy, 0.432) (autopsies, 0.430) (phonatory, 0.421) 
economic, (social, 0.574) (techno, 0.523) (organizational, 0.507) (sustainability, 0.500) (policies, 0.494) 
finance, (stores, 0.494) (know, 0.485) (firm, 0.459) (fulfilment, 0.432) (saying, 0.429) 
algebra, (algebras, 0.639) (eigenfunctions, 0.586) (commutative, 0.580) (gysin, 0.543) (theorem, 0.526) 
convergence, (guarantees, 0.464) (guaranteed, 0.452) (prove, 0.446) (posedness, 0.442) (neighborhood, 0.424) 
bacteria, (bacterial, 0.548) (assimilate, 0.440) (harbour, 0.438) (pathogenic, 0.433) (ascomycet

In [210]:
Ua = get_U(model.docvecs.doctag_syn0)

/home/ekvall/anaconda3/envs/kth-cluster/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  """Entry point for launching an IPython kernel.


In [211]:
authors = model.docvecs.index2entity
x2iA = {w:i for i, w in enumerate(authors)}
i2xA = {i:w for i, w in enumerate(authors)}

In [212]:
k = 15
for x in ["Kall, Lukas(u1gqsept)", "Savolainen, Peter", "Hudson, Paul"]:

    dd = cosine_similarity(Ua,Ua[x2iA[auth_to_id[str(x)]]].reshape(1,-1)).flatten()
    s = ''
    for i in np.argsort(dd)[::-1][:k + 1]:
        if i2xA[i] == auth_to_id[str(x)]: 
            continue

        if i2xA[i][0] == "u":
            s += '(%s, %.3lf) ' % (id_to_auth[i2xA[i]], dd[i])
    print('%s, %s' % (x, s))

Kall, Lukas(u1gqsept), (The, Matthew, 0.707) (Jeuken, Gustavo S., 0.494) (Ekeberg, Magnus, 0.470) (Algenas, Cajsa, 0.466) (Lundberg, Emma K., 0.464) 
Savolainen, Peter, (Oskarsson, Mattias, 0.795) (Klutsch, Cornelya F. C., 0.782) (Ardalan, Arman, 0.738) (Zhang, Ai-bing, 0.726) (Angleby, Helen, 0.688) (Arvestad, Lars, 0.582) (Dahlgren, Lars-Goran, 0.543) (Skollermo, Anna, 0.539) (Ekstrom, Camilla, 0.511) (Aspeborg, Henrik, 0.485) 
Hudson, Paul, (Anfelt, Josefine, 0.714) (Jahn, Michael, 0.691) (Shabestary, Kiyan, 0.683) (Karlsen, Jan, 0.628) (Kaczmarzyk, Danuta, 0.615) (Rockberg, Johan, 0.565) (Rostrup-Nielsen, J., 0.545) (Edfors, Fredrik, 0.540) (Forsstrom, Bjorn, 0.539) (Thomas, Quentin, 0.523) (Van Maris, Antonius J.A., 0.516) (Lundqvist, Magnus, 0.503) (Hallstrom, Bjorn M., 0.500) (Tegel, Hanna, 0.487) 


In [213]:
Ub = get_U(np.concatenate((model.docvecs.doctag_syn0, model.wv.vectors)))

/home/ekvall/anaconda3/envs/kth-cluster/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doctag_syn0` (Attribute will be removed in 4.0.0, use docvecs.vectors_docs instead).
  """Entry point for launching an IPython kernel.


In [214]:
both = authors + words
x2iB = {w:i for i, w in enumerate(both)}
i2xB = {i:w for i, w in enumerate(both)}

In [215]:
k = 25
for x in ["Kall, Lukas(u1gqsept)", "Savolainen, Peter", "Hudson, Paul"]:

    dd = cosine_similarity(Ub,Ub[x2iB[auth_to_id[str(x)]]].reshape(1,-1)).flatten()
    s = ''
    for i in np.argsort(dd)[::-1][:k + 1]:
        if i2xB[i] == x: continue
        try:
            int(i2xB[i][0])
        except:
            if not i2xB[i][0] == "u" and not i2xB[i][1] == "l":
                s += '(%s, %.3lf) ' % (i2xB[i], dd[i])
    print('%s, %s' % (x, s))

Kall, Lukas(u1gqsept), (percolator, 0.665) (shotgun, 0.631) (identifications, 0.602) (psms, 0.576) (peptides, 0.526) (rna, 0.478) (peptideatlas, 0.465) (proteome, 0.457) (crosstalkz, 0.443) 
Savolainen, Peter, (dogs, 0.888) (dog, 0.730) (mtdna, 0.703) (breeds, 0.673) (domestication, 0.618) (wolves, 0.597) (geographically, 0.568) (haplotypes, 0.564) (wolf, 0.555) (dingoes, 0.539) (yemen, 0.537) (asia, 0.536) 
Hudson, Paul, (synechocystis, 0.587) (cyanobacteria, 0.501) (biosynthesis, 0.491) (cerevisiae, 0.488) (prk, 0.467) (crispr, 0.466) (crispri, 0.460) (repress, 0.448) 
